In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [ ]:
HEADERS = ['State Name', 'District Name', 'Market Name', 'Variety', 'Group', 'Arrivals (Tonnes)', 'Min Price (Rs./Quintal)', 'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)', 'Reported Date']
df = pd.DataFrame(columns=HEADERS)

start = datetime(2020, 1, 1)
end = datetime(2025, 2, 17)
delta = timedelta(days=1)

while (start <= end):
    url = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=78&Tx_State=PB&Tx_District=0&Tx_Market=0&DateFrom={date}&DateTo={date}&Fr_Date={date}&To_Date={date}&Tx_Trend=2&Tx_CommodityHead=Tomato&Tx_StateHead=Punjab&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select--".format(date=start.strftime("%d-%b-%Y"))
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="tableagmark_new")
    if table is None:
        print(f"No data found for {start.strftime('%d-%b-%Y')}")
        start += delta
        continue
    rows = []
    for row in table.find_all("tr"):
        cells = []
        for td in row.find_all("td"):
            cells.append(td.text.strip())
        if cells:
            rows.append(cells)
    
    for row in rows:
        if len(row) == 10:
            df.loc[len(df)] = row
        
    start += delta

In [8]:
df['Reported Date'] = pd.to_datetime(df['Reported Date'], format='%d %b %Y', errors='coerce')

In [10]:
df.to_csv('../datasets/raw/agmarknet_punjab_potato_raw.csv', index=False, header=True)